In [1]:
# Compare avilable rest, emoid, and nback FC with what we already have

import pickle

seagate = '/run/media/anton/Seagate Portable Drive'
allts = None
meta = None

with open(f'{seagate}/PNC_PowerTS_float2.pkl', 'rb') as f:
    allts = pickle.load(f)
    
with open('/home/anton/Documents/Tulane/Research/PNC_Good/MegaMeta3.pkl', 'rb') as f:
    mega = pickle.load(f)
    
print([(task,len(list(tsdict.keys()))) for task,tsdict in allts.items()])
print(len(list(mega.keys())))

[('emoid', 928), ('nback', 907), ('rest', 877)]
941


In [12]:
# Get sub to response variable map
# New ts must be filtered
# 'wratStd'

qcfailedrest = [
    'sub-603403163265',
    'sub-605515760919',
    #'sub-600210241146'
]

import numpy as np
from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

tr = 1.83

X = []
age = []
sex = []

for i,pair in enumerate(mega.items()):
    key,value = pair
    if 'emoid' in value:
        key = 'sub-' + str(key)
        if key not in allts['emoid'] or key in qcfailedrest:
            print(f'Missing {key}')
            continue
        X.append(allts['emoid'][key])
        N = X[-1].shape[-1]
        X[-1] = butter_bandpass_filter(X[-1], [tr/20*N, 0.95*N], 2*N)
#         X.append(value['rest'])
        age.append(value['AgeInMonths']/12)
        if value['Gender'] == 'M':
            sex.append(0)
        else:
            sex.append(1)
        
X = np.stack(X)
age = np.array(age)
sex = np.array(sex)
sex2 = np.stack([sex == 0, sex == 1], axis=1)
        
print(X.shape)
print(age.shape)
print(sex.shape)
print(sex2.shape)

Missing sub-600135180939
Missing sub-605515760919
Missing sub-609664342210
(675, 264, 210)
(675,)
(675,)
(675, 2)


In [13]:
# Make pearson

ps = np.stack([np.corrcoef(X[i]) for i in range(X.shape[0])])
a,b = np.triu_indices(264, k=1)
ps = ps[:,a,b].reshape(ps.shape[0],-1)
print(ps.shape)

(675, 34716)


In [15]:
# Least squares regression on age

import torch
from math import floor

X_t = torch.from_numpy(ps).float().cuda()
y_t = torch.from_numpy(age).float().cuda()

# Training and test set

tot = []

for i in range(10):
    N = X_t.shape[0]
    a = floor(0.8*N)
    idcs = torch.arange(N)
    idcs = torch.randperm(N)
    trainIdcs = idcs[:a]
    testIdcs = idcs[a:]
    Xtr_t = X_t[trainIdcs]
    ytr_t = y_t[trainIdcs]
    Xt_t = X_t[testIdcs]
    yt_t = y_t[testIdcs]

    w, _, _, _ = torch.linalg.lstsq(Xtr_t, ytr_t)

    yhat = Xt_t@w
    rmse = torch.mean((yhat-yt_t)**2)**0.5
    tot.append(float(rmse))
    print(tot[-1])
    
print(sum(tot)/len(tot))

2.2294235229492188
2.527726173400879
2.3346707820892334
2.191744565963745
2.3130156993865967
2.554384231567383
2.4389212131500244
2.461623191833496
2.5651276111602783
2.380746603012085
2.3997383594512938


In [129]:
# Logistic regression on sex

from sklearn.linear_model import LogisticRegression
from math import floor

X_t = ps
y_t = sex

# Training and test set

tot = []

for i in range(10):
    N = X_t.shape[0]
    a = floor(0.8*N)
    idcs = np.arange(N)
    np.random.shuffle(idcs)
    trainIdcs = idcs[:a]
    testIdcs = idcs[a:]
    Xtr_t = X_t[trainIdcs]
    ytr_t = y_t[trainIdcs]
    Xt_t = X_t[testIdcs]
    yt_t = y_t[testIdcs]

    clf = LogisticRegression(max_iter=5000, C=1e1).fit(Xtr_t, ytr_t)
    yhat = clf.predict(Xt_t)
    acc = np.sum(yhat == yt_t)/yhat.shape[0]
    tot.append(float(acc))
    print(tot[-1])
    
print(sum(tot)/len(tot))

0.7586206896551724
0.7011494252873564
0.6954022988505747
0.7298850574712644
0.6954022988505747
0.7471264367816092
0.7471264367816092
0.7873563218390804
0.735632183908046
0.7183908045977011
0.7316091954022987


In [5]:
list(allts['rest'].keys())

['sub-600009963128',
 'sub-600018902293',
 'sub-600020927179',
 'sub-600031697545',
 'sub-600038720566',
 'sub-600039015048',
 'sub-600039665619',
 'sub-600045443700',
 'sub-600052596653',
 'sub-600053476714',
 'sub-600054124128',
 'sub-600059829137',
 'sub-600061763148',
 'sub-601053208894',
 'sub-601062830722',
 'sub-601064014005',
 'sub-601075042526',
 'sub-601078429870',
 'sub-601091916731',
 'sub-601097899414',
 'sub-601098855033',
 'sub-601110916219',
 'sub-601121451354',
 'sub-601127197796',
 'sub-601134535512',
 'sub-601141983954',
 'sub-601146803700',
 'sub-601150704931',
 'sub-601155992510',
 'sub-601160982029',
 'sub-601163770229',
 'sub-601167369441',
 'sub-601181364386',
 'sub-601194539197',
 'sub-600062084650',
 'sub-600262185931',
 'sub-600515650925',
 'sub-600700496428',
 'sub-600896033967',
 'sub-600994234394',
 'sub-601102119981',
 'sub-601197188395',
 'sub-601458344339',
 'sub-601834725907',
 'sub-602076199855',
 'sub-602238205398',
 'sub-601249791321',
 'sub-6012567